In [129]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten
from tensorflow.keras.regularizers import l2
import os
from os import listdir
import gdown
from zipfile import ZipFile
import pickle
import cv2
from os import listdir
from sklearn.preprocessing import LabelBinarizer

In [130]:
from tensorflow.keras.utils import img_to_array
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            img_gan = cv2.resize(image, (32,32))
            return img_to_array(img_gan)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [162]:
directory_root = 'Orignal_strawberry'
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    #print(root_dir)
    for plant_folder in root_dir :
        # remove .DS_Store from list
        if plant_folder == ".DS_Store" :
            root_dir.remove(directory)
            

    for plant_folder in root_dir :
#         print("on plant folder")
        plant_image_list = listdir(f"{directory_root}/{plant_folder}")
        
     
        
        for single_image in plant_image_list :
            
#             print("in plant folder")
            if single_image == ".DS_Store" :
                plant_image_list.remove(single_image)

        for image in plant_image_list[:1000]:
#             print("getting img path")
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True or image_directory.endswith(".png") == True or image_directory.endswith(".PNG") == True:
#                 print("checking if img correct format")
                gan_img = convert_image_to_array(image_directory)
                image_list.append(gan_img)
                label_list.append(plant_folder)
                    
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Image loading completed


In [163]:
# print(label_list)
label_gen = LabelBinarizer()
image_labels = label_gen.fit_transform(label_list)
pickle.dump(label_gen,open('label_transform.pkl', 'wb'))
n_classes = len(label_gen.classes_)
print(n_classes)
print(image_labels)

2
[[0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]]


In [164]:
new_image_list = np.array(image_list, dtype = np.float16) / 255.0
print(new_image_list.shape)

(2000, 32, 32, 3)


In [165]:
from sklearn.model_selection import train_test_split
print("making the training and testing split with 30% of the dataset as testing set")
x_train, x_test, y_train, y_test = train_test_split(new_image_list, image_labels, test_size=0.3, shuffle=True,random_state = 42) 
# x_train = x_train.astype('float32') / 255
# x_test = x_test.astype('float32') / 255
print(x_train.shape)
print(x_test.shape)

making the training and testing split with 30% of the dataset as testing set
(1400, 32, 32, 3)
(600, 32, 32, 3)


In [166]:
model = Sequential()
model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3, strides = 2,input_shape=(32,32,3)))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

# model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
# model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Flatten())
model.add(Dense(128,activation="relu"))

#Output layer
model.add(Dense(1,kernel_regularizer=l2(0.01),activation = "linear"))

In [167]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 512)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_33 (Dense)             (None, 1)               

In [168]:
model.compile(optimizer="adam",loss="squared_hinge", metrics = ['accuracy'])

In [169]:
result = model.fit(x_train,y_train,epochs=15)

Epoch 1/15
44/44 [==============================] - 1s 8ms/step - loss: 0.7865 - accuracy: 0.5807
Epoch 2/15
44/44 [==============================] - 0s 5ms/step - loss: 0.3147 - accuracy: 0.8579
Epoch 3/15
44/44 [==============================] - 0s 5ms/step - loss: 0.2360 - accuracy: 0.8893
Epoch 4/15
44/44 [==============================] - 0s 5ms/step - loss: 0.1614 - accuracy: 0.9250
Epoch 5/15
44/44 [==============================] - 0s 5ms/step - loss: 0.1649 - accuracy: 0.9343
Epoch 6/15
44/44 [==============================] - 0s 5ms/step - loss: 0.1527 - accuracy: 0.9321
Epoch 7/15
44/44 [==============================] - 0s 5ms/step - loss: 0.1217 - accuracy: 0.9450
Epoch 8/15
44/44 [==============================] - 0s 5ms/step - loss: 0.0925 - accuracy: 0.9521
Epoch 9/15
44/44 [==============================] - 0s 5ms/step - loss: 0.0650 - accuracy: 0.9700
Epoch 10/15
44/44 [==============================] - 0s 5ms/step - loss: 0.0632 - accuracy: 0.9721
Epoch 11/15
44/44 [

In [170]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_prob = model.predict(x_test)
y_pred = np.where(y_prob[:len(x_test)] >= 0.5, 1, 0)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.9266666666666666
Precision: 1.0
Recall: 0.8523489932885906
F1-score: 0.9202898550724637


In [161]:
model.save("plant_tomato_standAlone_BiGAN_svm.h5")